In [1]:
import pandas as pd
import regex as re
import fuzzywuzzy
import numpy as np
import geopandas as gpd
from fuzzywuzzy import fuzz

In [2]:
aeps = pd.read_excel(
    "./data/AEPSi Data for NSS.xlsx",
    2,
    usecols=[
        "Child ID",
        "Program Name",
        "AEPSi ID",
        "AssessID",
        "Gender",
        "Dev Status",
        "AEPS Level",
        "DOB",
        "Test Date",
        "Examiner",
        "Service Coordinator",
        "TEIS Point of Entry Office (POE)",
        "ESL",
        "County of Residence",
        "Number of Items",
        "fm_B4.0",
        "fm_B5.0",
        "cog_D2.0",
        "cog_E2.0",
        "cog_E4.0",
        "cog_F1.0",
        "cog_G1.0",
        "cog_G2.0",
        "cog_G3.0",
        "cog_G4.0",
        "cog_G5.0",
        "cog_G6.0",
        "sc_B1.0",
        "sc_B2.0",
        "sc_D1.0",
        "sc_D2.0",
        "sc_D3.0",
        "FM Raw Score",
        "FM Possible Score",
        "FM Percentage",
        "Cog Raw Score",
        "Cog Possible Score",
        "Cog Percentage",
        "SC Raw Score",
        "SC Possible Score",
        "SC Percentage"], 
    parse_dates = False) ## Big tall ugly command cuts down columns to just the ones we might need. 
aeps.head()

,Child ID,Program Name,AEPSi ID,AssessID,DOB,Gender,Dev Status,AEPS Level,Test Date,Examiner,...,sc_D3.0,FM Raw Score,FM Possible Score,FM Percentage,Cog Raw Score,Cog Possible Score,Cog Percentage,SC Raw Score,SC Possible Score,SC Percentage
0,11,Signal Centers,722747,10584304,2020-02-02 00:00:00,M,Dev. Delay or Disability,1,2022-04-21 00:00:00,Maria E. Vives-Rodriguez,...,0.0,51.0,66.0,0.7727,65.0,116.0,0.5603,52.0,92.0,0.5652
1,10724,Little Tennessee Valley Educational Cooperative,707878,10575064,2021-07-30 00:00:00,F,Dev. Delay or Disability,1,2022-03-01 00:00:00,Olivia J Woodall;Olivia Woodall,...,0.0,19.0,66.0,0.2879,26.0,116.0,0.2241,15.0,92.0,0.163
2,10724,Little Tennessee Valley Educational Cooperative,707878,10589703,2021-07-30 00:00:00,F,Dev. Delay or Disability,1,2022-05-09 00:00:00,Olivia J Woodall;Olivia Woodall,...,0.0,29.0,66.0,0.4394,41.0,116.0,0.3534,24.0,92.0,0.2609
3,11112,SRVS,665041,10535282,2020-04-27 00:00:00,F,Dev. Delay or Disability,1,2021-07-08 00:00:00,Beth Murphree,...,0.0,41.0,66.0,0.6212,36.0,116.0,0.3103,28.0,92.0,0.3043
4,11112,SRVS,665041,10503223,2020-04-27 00:00:00,F,Dev. Delay or Disability,1,2020-12-30 00:00:00,Beth Murphree,...,0.0,33.0,66.0,0.5000,18.0,116.0,0.1552,18.0,92.0,0.1957


In [3]:
aeps_temp = aeps
#aeps = aeps_temp
#Use as needed in testing to avoid re-importing the excel sheet. Avoids long datetime conversion.

In [4]:
aeps = aeps.applymap(lambda x: x.strip() if isinstance(x, str) else x) ## Strips all strings in dataframe
aeps = aeps.drop(aeps.index[17926:17959]) ## Removes empty and illogical child id's at end of dataframe
aeps = aeps.drop([0, 7, 1393, 17774]) ## Removes a few more illogical child id's and one row full of null values
aeps = aeps.drop_duplicates(subset='AssessID') ## Removes duplicate assessment results
aeps['child_id'] = aeps['Child ID'].astype(int)
aeps['dob'] = pd.to_datetime(aeps['DOB'], errors='coerce') ## errors='coerce' causes errors to be entered as 'NaT'
aeps['test_date'] = pd.to_datetime(aeps['Test Date'], errors='coerce')
aeps['age_at_test'] = aeps['test_date'] - aeps['dob']
aeps['test_number'] = aeps.groupby('child_id').age_at_test.rank()
aeps['examiner'] = aeps['Examiner'].str.replace(
    r"\;.*",""
    ).str.replace(
    r",.*",""
    ).str.replace(
    r"\/.*",""
    ).str.replace(
    r"\:",""
    ).str.lower()
aeps.head()

/tmp/ipykernel_38946/1568236774.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  aeps['examiner'] = aeps['Examiner'].str.replace(


,Child ID,Program Name,AEPSi ID,AssessID,DOB,Gender,Dev Status,AEPS Level,Test Date,Examiner,...,Cog Percentage,SC Raw Score,SC Possible Score,SC Percentage,child_id,dob,test_date,age_at_test,test_number,examiner
1,10724,Little Tennessee Valley Educational Cooperative,707878,10575064,2021-07-30 00:00:00,F,Dev. Delay or Disability,1,2022-03-01 00:00:00,Olivia J Woodall;Olivia Woodall,...,0.2241,15.0,92.0,0.163,10724,2021-07-30,2022-03-01,214 days,1.0,olivia j woodall
2,10724,Little Tennessee Valley Educational Cooperative,707878,10589703,2021-07-30 00:00:00,F,Dev. Delay or Disability,1,2022-05-09 00:00:00,Olivia J Woodall;Olivia Woodall,...,0.3534,24.0,92.0,0.2609,10724,2021-07-30,2022-05-09,283 days,2.0,olivia j woodall
3,11112,SRVS,665041,10535282,2020-04-27 00:00:00,F,Dev. Delay or Disability,1,2021-07-08 00:00:00,Beth Murphree,...,0.3103,28.0,92.0,0.3043,11112,2020-04-27,2021-07-08,437 days,3.0,beth murphree
4,11112,SRVS,665041,10503223,2020-04-27 00:00:00,F,Dev. Delay or Disability,1,2020-12-30 00:00:00,Beth Murphree,...,0.1552,18.0,92.0,0.1957,11112,2020-04-27,2020-12-30,247 days,2.0,beth murphree
5,11112,SRVS,665041,10485665,2020-04-27 00:00:00,F,Dev. Delay or Disability,1,2020-09-15 00:00:00,Beth Murphree,...,0.0776,6.0,92.0,0.0652,11112,2020-04-27,2020-09-15,141 days,1.0,beth murphree


In [6]:
#Start just by taking a look at the column
aeps.examiner

1         olivia j woodall
2         olivia j woodall
3            beth murphree
4            beth murphree
5            beth murphree
               ...        
17921       krystal prater
17922        doreen lawton
17923       krystal prater
17924    d'anitra reynolds
17925       ashley cordell
Name: examiner, Length: 17762, dtype: object

In [8]:
#Look at the value counts for the column
aeps.examiner.value_counts()

jen johnson                 547
karen julian                327
janet dixon                 298
kristen gilmartin talamo    239
michele valadie             233
                           ... 
jessica l mcbryar             1
laura harerll                 1
jessica beaver                1
ciara olive2                  1
marci hhopson                 1
Name: examiner, Length: 674, dtype: int64

In [10]:
#First, an easy adjustment, stripping any leading or lagging spaces
aeps.examiner = aeps.examiner.str.strip()
aeps.examiner

1         olivia j woodall
2         olivia j woodall
3            beth murphree
4            beth murphree
5            beth murphree
               ...        
17921       krystal prater
17922        doreen lawton
17923       krystal prater
17924    d'anitra reynolds
17925       ashley cordell
Name: examiner, Length: 17762, dtype: object

In [11]:
#We do see a decent reduction in the number of unique values (as indicated here by the number of lengths)
aeps.examiner.value_counts()

jen johnson                 559
karen julian                327
janet dixon                 298
kristen gilmartin talamo    239
michele valadie             233
                           ... 
hannah hamiltn                1
kscott                        1
jan apps                      1
jennifer  cavitt              1
marci hhopson                 1
Name: examiner, Length: 653, dtype: int64

In [12]:
#Now we'll look for any cells that contain a semicolon and keep only what is to the left of the semicolon
aeps.examiner = aeps.examiner.str.extract('(^[^;]+)')
aeps.examiner

1         olivia j woodall
2         olivia j woodall
3            beth murphree
4            beth murphree
5            beth murphree
               ...        
17921       krystal prater
17922        doreen lawton
17923       krystal prater
17924    d'anitra reynolds
17925       ashley cordell
Name: examiner, Length: 17762, dtype: object

In [13]:
#This leads to another reduction in unique values
aeps.examiner.value_counts()

jen johnson                 559
karen julian                327
janet dixon                 298
kristen gilmartin talamo    239
michele valadie             233
                           ... 
hannah hamiltn                1
kscott                        1
jan apps                      1
jennifer  cavitt              1
marci hhopson                 1
Name: examiner, Length: 653, dtype: int64

In [15]:
#Making the value counts into a dataframe
ex_counts = aeps.examiner.value_counts().to_frame()
#Keeping only those entries that occur more than once
ex_counts = ex_counts.loc[ex_counts.examiner > 1]
#Turning that into a list
examiner_list =ex_counts.index.to_list()
examiner_list

['jen johnson',
 'karen julian',
 'janet dixon',
 'kristen gilmartin talamo',
 'michele valadie',
 'kandace haag',
 'clayton rouse',
 'hannah vantrease',
 'traci wilber',
 'aimee loeffler',
 'larendi armstrong',
 'david fosbinder',
 'lori boles',
 'esmeralda',
 'shannon blakeman',
 'molly morris',
 'ashley cordell',
 'marisa borreca',
 'angel early',
 'angie mccoy',
 'teresa motley',
 'laura harrell',
 'camille crawford',
 'gina wilson',
 'barbara may',
 'jennifer cavitt',
 'leeann finnegan',
 'sarah watkins',
 'kristen gilmartin',
 'hannah hamilton',
 'sarah dugan',
 'mallory sigler',
 'laura riddell',
 'leslie throneberry',
 'morgan blake',
 'diana barrick',
 'patricia reese',
 'amy trent',
 'deanna stephens',
 'michelle fox',
 'cindy kennard',
 'virginia martino',
 'megan jones',
 'rachel stewart',
 'kristina scott',
 'rosario langlois',
 'karisha mcneal',
 'jessie bradley',
 'regina piercy',
 'lisa ketner',
 'jennifer long',
 'britney mcconnell',
 'kourtney daniels',
 'melissa moor

In [17]:
#creating a column to indicate whether or not a name is changed (Y=Yes, N=No); this block contributed by instructor Chris Wright
aeps['changed'] = 'N'

#Building a for loop that compares the entries in the examiner column to the entries in our list
for name in examiner_list:
    
        for ind, row in aeps.iterrows():
            #For any row that indicates it has NOT be changed, the following happens
            if row.changed == 'N':
                #Create a ration of how similar the entry in the dataframe is to the entry in the list
                ratio = fuzz.token_set_ratio(name, row.examiner)
                #For any entries that have a ratio of 100, meaning they match exactly...
                if ratio == 100:
                    #Go ahead and mark the 'changed' column as Y so that row won't be looked at again
                    aeps.loc[ind, 'changed'] = 'Y'
                #For  any entries that are very similar (ratio > 88) but not exactly the same...
                if ((ratio > 88)  & (ratio < 100)):
                    #go ahead and change the entry in the dataframe to match the entry in the list
                    aeps.loc[ind, 'examiner'] = name
                    #and mark the 'changed' column as Y so that row won't be evaluated again
                    aeps.loc[ind, 'changed'] = 'Y'
#This loop continues until all rows are changed or have no match that creates a ratio greater than 88            
        

In [18]:
#Take a look at how many unique values remain
aeps.examiner.nunique()

556

In [19]:
#look at how many rows were marked Y and N after running our loop
aeps['changed'].value_counts()

Y    17725
N       37
Name: changed, dtype: int64

In [20]:
#look at the updated value counts for Examiner names
aeps.examiner.value_counts()

jen johnson                 562
karen julian                329
janet dixon                 298
kristen gilmartin talamo    241
michele valadie             236
                           ... 
gisele                        1
amy j trent                   1
candace peacock               1
kaley rogers                  1
morgan haas                   1
Name: examiner, Length: 556, dtype: int64

In [21]:
#Get rid of our 'changed' column now that it's served its purpose
aeps = aeps.drop(columns = 'changed')

In [27]:
#save the cleaned up df to a csv
aeps.to_csv('data/cleaned_AEPS.csv', index = False)

In [ ]:
#read in the new csv to check it, note the warning giving you a heads up about some columns with mixed datatypes
#new = pd.read_csv('../data/cleaned_AEPS.csv')

In [ ]:
#new

In [22]:
pd.set_option('display.max_rows', None)
print(aeps.examiner.value_counts())
pd.set_option('display.max_rows', 10)

jen johnson                 562
karen julian                329
janet dixon                 298
kristen gilmartin talamo    241
michele valadie             236
kandace haag                224
clayton rouse               199
hannah vantrease            193
traci wilber                192
aimee loeffler              171
larendi armstrong           143
david fosbinder             138
lori boles                  137
esmeralda                   135
shannon blakeman            126
molly morris                125
ashley cordell              123
angel early                 116
marisa borreca              114
angie mccoy                 110
laura harrell               106
teresa motley               106
camille crawford            102
gina wilson                 100
barbara may                  95
jennifer cavitt              94
leeann finnegan              94
hannah hamilton              93
kristen gilmartin            92
sarah watkins                92
mallory sigler               91
sarah du

In [23]:
chris = pd.read_csv('../TEIS Partner Project/cleaned_AEPS.csv')

/tmp/ipykernel_38946/1909226299.py:1: DtypeWarning: Columns (0,3,279,282,285,288) have mixed types. Specify dtype option on import or set low_memory=False.
  chris = pd.read_csv('../TEIS Partner Project/cleaned_AEPS.csv')


In [24]:
chris.shape

(17959, 307)

In [26]:
pd.set_option('display.max_rows', None)
print(chris.Examiner.value_counts())
pd.set_option('display.max_rows', 10)

Jen Johnson                       552
Karen Julian                      332
Janet Dixon                       300
Kristen Gilmartin Talamo          241
Michele Valadie                   240
Kandace Haag                      225
Clayton Rouse                     199
Traci Wilber                      192
Hannah Vantrease                  192
Aimee Loeffler                    173
LaRendi Armstrong                 143
David Fosbinder                   141
Lori Boles                        137
esmeralda                         131
Shannon Blakeman                  127
Molly Morris                      126
Ashley Cordell                    122
Barbara May                       121
Angel Early                       120
Marisa Borreca                    114
Angie McCoy                       110
Teresa Motley                     107
Laura Harrell                     107
Gina Wilson                       101
LeeAnn Finnegan                    95
Jennifer Cavitt                    94
Hannah Hamil